In [74]:
import os
import sys
sys.path.append(os.path.abspath('../tapeagents'))

from tapeagents.llms import OpenrouterLLM
from tapeagents.agent import Agent, Node
from tapeagents.core import Prompt
from tapeagents.dialog_tape import DialogTape, UserStep
from tapeagents.orchestrator import main_loop
import json

# API Configuration
api_key = "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"
llm = OpenrouterLLM(
    model_name="meta-llama/llama-3.3-70b-instruct:free",
    api_token=api_key,
    parameters={"temperature": 0.1},
)

In [75]:
from tapeagents.environment import ToolCollectionEnvironment
from tapeagents.tools.calculator import Calculator


environment = ToolCollectionEnvironment(tools=[ Calculator()])
environment.initialize()
print(environment.tools_description())

- Calculator - Tool to evaluate math expressions


In [76]:
# Load regulation once
regulation = open("Synthetic Childcare Subsidy Regulation.md").read()

In [64]:
from tapeagents.nodes import StandardNode, Stop

system_prompt = f"""You are a childcare subsidy decision summary agent. Your job is to create clear, professional summaries of subsidy decisions that have already been made by a previous decision-making agent.

## CHILDCARE SUBSIDY REGULATION:
{regulation}

You will receive:
- Application data
- Eligibility assessment (the final decision)
- Validation flags (issues or criteria that were flagged)

## YOUR TASK FOR EACH CASE:
Create a professional summary of this childcare subsidy decision including:
1. **Case Overview**: Brief description of applicant and case
2. **Final Decision**: State the eligibility assessment clearly
3. **Validation Flags Explanation**: Explain what each validation flag means in plain language
4. **Key Factors**: Main factors that influenced the decision based on the flags
5. **Financial Impact**: If eligible, mention subsidy calculations
6. **Administrative Notes**: Important procedural information

## IMPORTANT GUIDELINES:
1. Summarizing the eligibility assessment clearly
2. Explaining validation flags in plain language and how they influenced the decision
3. Do NOT re-evaluate or change the eligibility assessment - only summarize what was already decided."""

agent = Agent.create(
    llms=llm,
    nodes=[
        StandardNode(
            name="summary",
            system_prompt=system_prompt,
            guidance="Review the decision data and flags provided, then create a professional summary of the childcare subsidy decision. Focus on clarity and completeness for official documentation.",
        ),
        Stop(),
    ],
    known_actions=environment.actions(),
    tools_description=environment.tools_description(),
)

In [51]:
with open("processed_applications.json") as f:
    processed_applications = json.load(f)

In [77]:

summaries = []

for i, app in enumerate(processed_applications):
    print(f"\n{'='*50}")
    print(f"Processing Application {i+1}/{len(processed_applications)}")
    print(f"{'='*50}")
    
    eligibility_decision = app.get("eligibility_assessment", "No decision found")
    validation_flags = app.get("validation_flags", [])
    case_id = app.get("application_id", f"CASE-{i:04d}")
    
    print(f"📋 Case ID: {case_id}")
    print(f"⚖️  Decision: {eligibility_decision}")
    print(f"🚩 Flags: {', '.join(validation_flags) if validation_flags else 'No flags'}")

        # Create user message for this specific application
    user_message = f"""

## APPLICATION DATA:
{json.dumps(app, indent=2)}

## ELIGIBILITY DECISION:
{eligibility_decision}

## VALIDATION FLAGS:
{", ".join(validation_flags) if validation_flags else "No validation flags"}

"""
    tape = DialogTape(steps=[UserStep(content=user_message)])
    
    for event in main_loop(agent, tape, environment):
        if event.agent_event and event.agent_event.step:
            step = event.agent_event.step
            print(f"Agent step ({step.metadata.node}):\n{step.llm_view()}\n---")
        elif event.observation:
            print(f"Tool call result:\n{event.observation.short_view()}\n---")
        elif event.agent_tape:
            tape = event.agent_tape

    # ✅ COLLECT SUMMARY INSIDE THE LOOP - for each application
    try:
        if hasattr(tape[-2], 'reasoning'):
            summary = tape[-2].reasoning
        elif hasattr(tape[-1], 'reasoning'):
            summary = tape[-1].reasoning
        else:
            summary = str(tape[-1])
            
        summaries.append({
            "case_id": case_id,
            "summary": summary,
        })
        print(f"✅ Summary for Application {i+1} ({case_id}):")
        print(summary)
        print("-" * 50)
        
    except Exception as e:
        print(f"❌ Error processing {case_id}: {e}")
        summaries.append({
            "case_id": case_id,
            "error": str(e),
        })



Processing Application 1/40
📋 Case ID: A001
⚖️  Decision: REQUIRES_REVIEW
🚩 Flags: inconsistent_data_format, missing_required_fields, income_threshold_exceeded, employment_status_invalid, child_age_inconsistency, high_hours_request, documentation_incomplete, human_review_required
Agent step (summary):
## Case Overview:
The applicant, with application ID A001, has submitted a request for childcare subsidy. The household consists of one child aged 3, with the applicant being unemployed and the partner being employed. The applicant has requested 137 hours of childcare.

## Final Decision:
The eligibility assessment for this application is currently marked as "REQUIRES_REVIEW". This indicates that the initial automated assessment has identified factors that require a more detailed examination by a reviewer to determine the applicant's eligibility for the childcare subsidy.

## Validation Flags Explanation:
Several validation flags have been raised in this application:
- **income_threshold

In [78]:
# Save all summaries after the loop completes
with open("decision_summaries.json", "w") as f:
    json.dump(summaries, f, indent=2)

print(f"\n🎉 Processing complete!")
print(f"Total applications: {len(processed_applications)}")
print(f"Successful summaries: {len([s for s in summaries if 'error' not in s])}")
print(f"Errors: {len([s for s in summaries if 'error' in s])}")


🎉 Processing complete!
Total applications: 40
Successful summaries: 40
Errors: 0
